In [4]:
import pyhdf.SD as sds
import pandas as pd
import os
from datetime import datetime
from IPython.display import clear_output

# 7,5 km
#Manaus_EMBRAPA -2.954minlat -2.828maxlat -60.033minlong -59.907maxlong
#Rio Branco (9.957S, 67.869W) -10.02minlat -9.894maxlat -67.932minlong -67.806maxlong
#Ji_Parana_SE (10.934S, 61.852W) -10.997minlat -10.871maxlat -61.915minlong -61.789maxlong

# 12,5 km
#Manaus_EMBRAPA (2.9S, 60.0W) -3.0123minlat -2.7877maxlat -60.1124minlong -59.8876maxlong
#Rio Branco (9.957S, 67.869W) -10.0693minlat  -9.8447maxlat -67.9829minlong -67.7551maxlong
#Ji_Parana_SE (10.934S, 61.852W) -11.0463minlat -10.8217maxlat -61.9664minlong -61.7376maxlong

# 17,5 km


# 22,5 km


# 27,5 km

min_lat = -3.0123
max_lat = -2.7877
min_long = -60.1124
max_long = -59.8876

folder_path = "/run/media/lucas/TOSHIBA EXT/Arquivos_MODIS/MOD043K/"

files = [file for file in os.listdir(folder_path) if file.endswith('.hdf')]
n_files = len(files)
n_files

32067

In [5]:
output_values = []
counter = 0
for file in files:
    file_path = os.path.join(folder_path, file)

    hdf = sds.SD(file_path, sds.SDC.READ)

    AOD = hdf.select('Optical_Depth_Land_And_Ocean')[:]
    Longitude = hdf.select('Longitude')[:]
    Latitude = hdf.select('Latitude')[:]

    df = pd.DataFrame()
    df['AOD'] = AOD.flatten()
    df['Longitude'] = Longitude.flatten()
    df['Latitude'] = Latitude.flatten()

    filtered_df = df.copy()
    filtered_df = df[df['AOD'] != -9999]
    filtered_df = filtered_df.query('Latitude >= @min_lat and Latitude <= @max_lat and Longitude >= @min_long and Longitude <= @max_long')

    mean_aod = filtered_df.AOD.mean() * 0.001
    output_values.append(mean_aod)

    clear_output()
    counter+=1
    print(f"{counter} of {n_files} done.")


32067 of 32067 done.


In [6]:
def julian_with_year_to_date(julian_date_with_year):
    return datetime.strptime(str(julian_date_with_year), '%Y%j').date()

dates = []

for f in files:
    year = f[10:14]
    day = f[14:17]
    dates.append(julian_with_year_to_date(f'{year}{day}'))

In [7]:
final_df = pd.DataFrame()

final_df['date'] = dates
final_df['AOD'] = output_values

final_df.to_csv('MOD043ktemporal_serie_Manaus_Embrapa_aod2010_2023.csv')

In [9]:
df = pd.read_csv("MOD043ktemporal_serie_Manaus_Embrapa_aod2010_2023.csv")
df.tail()

,Unnamed: 0,date,AOD
32062,32062,2023-07-29,NaN
32063,32063,2023-07-29,0.132543
32064,32064,2023-07-29,NaN
32065,32065,2023-07-29,NaN
32066,32066,2023-07-29,NaN
